In [1]:
from datasets import load_dataset
from dataclasses import dataclass, field, fields    ## For TrlParser

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    set_seed
)
from trl import SFTTrainer, SFTConfig, TrlParser, setup_chat_format
from peft import LoraConfig

from sklearn.model_selection import train_test_split

import logging
import torch

import os
import json
import random
import numpy as np

/root/miniconda3/envs/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/LLM/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/root/miniconda3/envs/LLM/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value T

In [2]:
def timer(func):
    """
    함수 실행 시간 출력
    """
    import time
    import datetime

    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()

        sec = end - start
        worktime = str(datetime.timedelta(seconds=sec)).split(".")[0]
        print(f"Working Time: {worktime}")
        return result

    return wrapper

In [3]:
train_ds = load_dataset("json", data_files = "train_dataset.json", split = "train")
test_ds = load_dataset("json", data_files = "test_dataset.json", split = "train")

## 토크나이저 로드 및 설정
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    use_fast = True,            ## Rust로 구현된 Fast Tokenizer 사용 (Qwen, RoPE, ChatGLM 등의 특이한 구조에서는 호환 안됨)
    trust_remote_code = True)   ## 모델 코드 전체 다운로드 후 사용
tokenizer.pad_token = tokenizer.eos_token       ## 패딩할 토큰 설정
tokenizer.padding_side = "left"                 ## 디코더이므로 왼쪽을 패딩 (마지막 토큰을 보고 생성)


In [4]:
LLAMA_3_CHAT_TEMPLATE = (
    "{{ bos_token }}"
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{ '<|start_header_id|>system<|end_header_id|>\n\n' + message['content'] + eos_token }}"
        "{% elif message['role'] == 'user' %}"
            "{{ '<|start_header_id|>user<|end_header_id|>\n\n' + message['content'] +  eos_token }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '<|start_header_id|>assistant<|end_header_id|>\n\n'}}"
            "{% generation %}"
            "{{ message['content'] +  eos_token }}"
            "{% endgeneration %}"
        "{% endif %}"
    "{% endfor %}"
    "{%- if add_generation_prompt %}"
    "{{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}"
    "{%- endif %}"
)

In [5]:
tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE

In [22]:
train_ds[0]["messages"]

[{'content': '당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.',
  'role': 'system'},
 {'content': '생선찌개를 비린내 없이 끓이는 방법은 무엇인가요?', 'role': 'user'},
 {'content': '생선찌개를 맛있게 끓이는 방법으로 비린내를 없애는 방법이 있습니다. 아래 방법들을 참고해보세요. \n\n- 비린내가 많이 나는 생선찌개에 마지막으로 식초를 넣으면 비린내가 없어집니다. 또한, 생선을 구울 때 껍질에 식초를 바르면 껍질이 벗겨지지 않고 제 모양대로 구울 수 있습니다. \n- 생선찌개를 만들 때 생선이 다 익은 다음 된장을 풀어 넣으면 비린내를 없앨 수 있습니다. \n- 깨끗이 손질한 생선이라도 미처 손질하지 못한 잡티가 붙어 있을 수 있는데, 끓이기 전에 팔팔 끓는 물을 살짝 끼얹으면 비린내도 가시고 국물이 깔끔합니다. \n- 간을 한 국물이 한참 끓으면 그때 생선을 넣습니다. \n\n위 방법을 참고해서 집에서 맛있는 비린내 없는 생선찌개를 만들어 보세요.',
  'role': 'assistant'}]

In [23]:
print(tokenizer.apply_chat_template(train_ds[0]["messages"], tokenize = False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

생선찌개를 비린내 없이 끓이는 방법은 무엇인가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

생선찌개를 맛있게 끓이는 방법으로 비린내를 없애는 방법이 있습니다. 아래 방법들을 참고해보세요. 

- 비린내가 많이 나는 생선찌개에 마지막으로 식초를 넣으면 비린내가 없어집니다. 또한, 생선을 구울 때 껍질에 식초를 바르면 껍질이 벗겨지지 않고 제 모양대로 구울 수 있습니다. 
- 생선찌개를 만들 때 생선이 다 익은 다음 된장을 풀어 넣으면 비린내를 없앨 수 있습니다. 
- 깨끗이 손질한 생선이라도 미처 손질하지 못한 잡티가 붙어 있을 수 있는데, 끓이기 전에 팔팔 끓는 물을 살짝 끼얹으면 비린내도 가시고 국물이 깔끔합니다. 
- 간을 한 국물이 한참 끓으면 그때 생선을 넣습니다. 

위 방법을 참고해서 집에서 맛있는 비린내 없는 생선찌개를 만들어 보세요.<|eot_id|>


In [6]:
## 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,                    ## 4비트 양자화
    bnb_4bit_use_double_quant = True,       ## 추가 양자화로 성능 손실 없이 파라미터당 0.4bit 추가 절약
    bnb_4bit_quant_type = "nf4",            ## 양자화 데이터 타입 지정: 4비트 기반 모델 훈련 시 사용
    bnb_4bit_compute_dtype = torch.bfloat16 ## Llama-3.1-8B의 학습 자료형. 저장은 4비트지만, 계산은 양자화 없이
)

## 모델 로드 및 설정
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    device_map = "cuda:0",
    use_cache = False,                          ## VRAM 캐시 미사용, 추론 속도 저하. gradienc_checkpointing과 동시 사용 불가
    low_cpu_mem_usage = True,                   ## CPU RAM 사용량 적게...
    attn_implementation = "flash_attention_2",  ## flash_attention 연산 사용
    quantization_config = bnb_config,
    dtype = torch.bfloat16                      ## Llama-3.1-8B의 자료형으로 설정
)

model.gradient_checkpointing_enable()

Loading checkpoint shards: 100%|██████████| 4/4 [00:20<00:00,  5.24s/it]


In [16]:
def formatting_and_masking(example):
    all_input_ids = []
    all_labels = []
    
    for i, message in enumerate(example["messages"]):
        current_messages = [message]
        encoded_tensor = tokenizer.apply_chat_template(
            current_messages,
            tokenize = True,
            add_generation_prompt = False,
            return_tensors = "pt"
        )

        input_ids = encoded_tensor.squeeze(0).tolist()

        # print(f"Original Message: {current_messages}")
        # print(f"Apply chat template: {input_ids}")

        if message["role"] == "assistant":
            labels = list(input_ids)           ## 생성부는 그대로
        else:
            labels = [-100] * len(input_ids)   ## system, user는 마스킹

        # print(f"After Masking: {labels}")

        all_input_ids.extend(input_ids)
        all_labels.extend(labels)

    return {
        "input_ids": all_input_ids,
        "labels": all_labels,
        "attention_mask": [1]*len(all_input_ids)
    }

In [9]:
peft_config = LoraConfig(
    r = 64,
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

training_args = SFTConfig(
    max_length = 1024,
    output_dir = "./results/assistant-only-4-epoch",
    report_to = "none",
    assistant_only_loss = True,
    learning_rate = 5e-5,
    lr_scheduler_type = "cosine_with_restarts",
    lr_scheduler_kwargs = {"num_cycles": 3},
    num_train_epochs = 4,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 4,
    optim = "adamw_torch_fused",
    logging_steps = 100,
    save_strategy = "epoch",
    weight_decay = 0.01,
    max_grad_norm = 0.5,
    warmup_ratio = 0.03,
    bf16 = True,
    tf32 = True,
    gradient_checkpointing = True,
    packing = True,
    dataloader_num_workers = 4,
    push_to_hub = True,
    dataset_kwargs = {
        "add_special_tokens": False,
        "append_concat_token": False
    }
)

trainer = SFTTrainer(
    model = model,
    args = training_args,
    train_dataset = train_ds,
    eval_dataset = test_ds,
    processing_class = tokenizer,
    peft_config = peft_config
)

/root/miniconda3/envs/LLM/lib/python3.12/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/root/miniconda3/envs/LLM/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Packing eval dataset: 100%|██████████| 2116/2116 [00:00<00:00, 26918.20 examples/s]


In [30]:
next(iter(trainer.train_dataset))["assistant_masks"][:100]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [12]:
# trainer의 collator 가져오기
collator = trainer.data_collator

# 원본 샘플 하나를 collate 시도
batch = collator([trainer.train_dataset[0]])

In [11]:
print(tokenizer.decode(next(iter(trainer.train_dataset))["input_ids"]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

과일을 이용한 피부 미용은 어떤 것이 있는지 알려주세요. 각 과일별로 어떻게 활용할 수 있고, 어디에 좋은 효과가 있는지도 알려주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

과일로 할 수 있는 피부 미용은 매우 다양합니다. 각 과일마다 그에 맞는 방법으로 활용할 수 있습니다. 따라서 각 과일에 대한 정보와 활용법을 알아보겠습니다.

- 레몬: 피부를 맑고 희게하고 모세혈관을 튼튼하게 해 기미, 주근깨, 빨간 뾰루지 등에 좋습니다. 산도가 강하므로 다른 재료와 섞어 사용하는 것이 좋습니다. 레몬즙 1작은술에 우유 약간, 해조 가루 약간을 섞어 부드럽게 만든 후 사용합니다.

- 키위: 비타민 C가 풍부해서 미백 효과가 뛰어나며, 당분, 무기질, 미네랄, 철분 등도 풍부해 탄력을 좋게 하고 피부의 수분 함유량을 높여 줍니다. 키위 반개를 갈아 오트밀 가루를 넣고 걸쭉해지면 사용합니다.

- 바나나: 보습 효과가 뛰어나며 메마른 피부에 좋습니다. 환절기 건조 피부에도 좋으며, 잔주름이 잘 생기는 악건성 피부에도 사용할 수 있습니다. 으깬 바나나 1큰술에 달걀 노른자와 밀가루를 섞어 사용합니다.

- 딸기: 피부를 희게

In [22]:
print(tokenizer.decode(next(iter(trainer.train_dataset))["input_ids"]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

당신은 다양한 분야의 전문가들이 제공한 지식과 정보를 바탕으로 만들어진 AI 어시스턴트입니다. 사용자들의 질문에 대해 정확하고 유용한 답변을 제공하는 것이 당신의 주요 목표입니다. 복잡한 주제에 대해서도 이해하기 쉽게 설명할 수 있으며, 필요한 경우 추가 정보나 관련 예시를 제공할 수 있습니다. 항상 객관적이고 중립적인 입장을 유지하면서, 최신 정보를 반영하여 답변해 주세요. 사용자의 질문이 불분명한 경우 추가 설명을 요청하고, 당신이 확실하지 않은 정보에 대해서는 솔직히 모른다고 말해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  

토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 빛의 궤적으로 보이는 건가요? 또한, 야간에 빠르게 움직이는 자동차를 장노출로 찍었을 때 빛의 궤적이 생기는 것과 같은 원리일까요? 그리고 빛의 궤적이 생기는 것은 우주라는 어두운 환경 특성 때문이라고 생각됩니다. 이게 맞을까요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카시니 탐사선에서 찍은 고해상도 사진에서 빛의 띠가 보이는 것입니다.  

실제로는 토성의 고리 입자들의 운동이 장노출 사진에서 잔상이 생기는 이유와 관련이 없습니다. 물체의 운동은 토성의 고리가 매끄럽게 보이는 이유와 상관이 없습니다. 

밀도 차이로 생긴 미세한 입자들의 밀도는 연속적인 것이 아니며 광학계의 분